In [1]:
import math
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import plotly.express as px
import time
import os
pd.options.plotting.backend = "plotly"

from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend

from dataclasses import dataclass
from typing import TypedDict, List, Callable, NamedTuple
import copy
from dataclasses import field
from functools import partial
from enum import Enum
# from model.types import * 


#types 

PERCENTAGE = float
USD = float
USD_PER_MONTH = float
MONTHS = int
MULTIPLIER = float
USD_PER_TOKEN = float
TOKENS = int

In [2]:

DELTA_TIME=1
Run = 1
Timestep = 100

def default(obj):
    return field(default_factory=lambda: copy.copy(obj))




@dataclass
class Parameters:
    """Params for the model"""

    # Fundraise and runway calcs
    raise_amount : USD = default([5_000_000])
    

    # Token Generation and sale calcs
    total_token_supply : TOKENS = default([180_000_000])

    private_price : USD_PER_TOKEN = default([0.1])
    public_price : USD_PER_TOKEN = default([0.12])

    private_raise : USD = default([4_800_000])
    public_raise : USD = default([200_000])

# Initialize Parameters instance with default values
system_params = Parameters().__dict__

In [3]:
@dataclass
class Agents:
    type: str = ''
    InvestorBool: bool = False
    unlocked_at_listing: PERCENTAGE = 0.0
    lock_term_after_listing: MONTHS = 0
    release_term: MONTHS = 0
    perc_total_token_supply: PERCENTAGE = 0.0
    tokens: TOKENS = 0

In [4]:
# state variables



private_round_size = system_params['private_raise'][0]/system_params['private_price'][0]
public_round_size = system_params['public_raise'][0]/system_params['public_price'][0]



Investor_list = [Agents(type='Private', InvestorBool=True, unlocked_at_listing=25.0, lock_term_after_listing=3, release_term=24, 
                        perc_total_token_supply= np.round(private_round_size/system_params['total_token_supply'][0],3), tokens=0).__dict__,
                 Agents(type='Public', InvestorBool=True, unlocked_at_listing=0.0, lock_term_after_listing=0, release_term=6, 
                        perc_total_token_supply= np.round(public_round_size/system_params['total_token_supply'][0],3), tokens=0).__dict__,
                 Agents(type='Team', InvestorBool=False, unlocked_at_listing=0.0, lock_term_after_listing=9, release_term=36, 
                        perc_total_token_supply= 0.15, tokens=0).__dict__,
                 Agents(type='Liquidity', InvestorBool=False, unlocked_at_listing=100.0, lock_term_after_listing=0, release_term=12, 
                        perc_total_token_supply= 0.05, tokens=0).__dict__,
                 Agents(type='Ecosystem', InvestorBool=False, unlocked_at_listing=0.0, lock_term_after_listing=0, release_term=36, 
                        perc_total_token_supply= 0.10, tokens=0).__dict__,]



def correct_perc_total_token_supply(Investor_list):
    total = 0
    for agent in Investor_list:
        if agent['type']!='Ecosystem':
            total += agent['perc_total_token_supply']
    for agent in Investor_list:
        if agent['type']=='Ecosystem':
            agent['perc_total_token_supply'] = np.round(1-total,3)
    return None

correct_perc_total_token_supply(Investor_list)

@dataclass
class StateVariables:
    # Investors
    Investors: List[Agents] = default(Investor_list)
    # Total: int= 0
    # Total_as_perc_of_total_supply: PERCENTAGE = 0.0
    # FLAG: bool = False
    # MarketCap_at_highest_round_price: USD = 0.0
    # Market_cap_at_1x_highest_round_valuation: USD = 0.0


initial_state = StateVariables().__dict__

In [5]:
Investor_df = pd.DataFrame(Investor_list)
Investor_df['perc_total_token_supply']=Investor_df['perc_total_token_supply']*100
# Investor_df

In [6]:
def p_monthly_unlocks(params, substep, state_history, prev_state, **kwargs):
    agents = prev_state['Investors']
    new_tokens = []
    for agent in agents:
        if prev_state['timestep'] == agent['lock_term_after_listing']:
            new_tokens.append(agent['unlocked_at_listing']*params['total_token_supply']*agent['perc_total_token_supply']/100)
        elif (prev_state['timestep'] > agent['lock_term_after_listing'] and (prev_state['timestep'] <= (agent['lock_term_after_listing']+agent['release_term']))):
            # new_tokens.append(((agent['perc_total_token_supply'])*params['total_token_supply'])/agent['release_term'])
            new_tokens.append((agent['perc_total_token_supply']*(1-(agent['unlocked_at_listing']/100)))*params['total_token_supply']/agent['release_term'])

        else:
            new_tokens.append(0)
    
    return {'new_unlocks': np.round(new_tokens)}

def s_update_tokens(params, substep, state_history, prev_state, policy_input, **kwargs):
    agents=prev_state['Investors']
    new_tokens = policy_input['new_unlocks']
    for i, agent in enumerate(agents):
        agent['tokens'] += new_tokens[i]
    return ('Investors', agents)

# def s_update_total(params, substep, state_history, prev_state, policy_input, **kwargs):
#     new_tokens = policy_input['new_unlocks']
#     total = prev_state['Total'] + sum(new_tokens)
#     return ('Total', total)

# def s_update_total_perc(params, substep, state_history, prev_state, policy_input, **kwargs):
#     total = prev_state['Total'] + sum(policy_input['new_unlocks'])
#     total_as_perc_of_total_supply = total/params['total_token_supply']
#     return ('Total_as_perc_of_total_supply', total_as_perc_of_total_supply)


state_update_blocks = [
    {
        'policies': {
            'p_monthly_unlocks': p_monthly_unlocks,
        },
        'variables': {
            # 'Total': s_update_total,
            'Investors': s_update_tokens,
            # 'Total_as_perc_of_total_supply': s_update_total_perc,

        }
        
    },

]




In [7]:
# config and run

TIMESTEPS = 120
RUNS = 1


model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=system_params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

# simulation.model.params.update({
#     '': [
        # 5
#     ]
# })

experiment = Experiment(simulation)
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()


df = pd.DataFrame(result)
df

,Investors,simulation,subset,run,substep,timestep
0,"[{'type': 'Private', 'InvestorBool': True, 'un...",0,0,1,0,0
1,"[{'type': 'Private', 'InvestorBool': True, 'un...",0,0,1,1,1
2,"[{'type': 'Private', 'InvestorBool': True, 'un...",0,0,1,1,2
3,"[{'type': 'Private', 'InvestorBool': True, 'un...",0,0,1,1,3
4,"[{'type': 'Private', 'InvestorBool': True, 'un...",0,0,1,1,4
...,...,...,...,...,...,...
116,"[{'type': 'Private', 'InvestorBool': True, 'un...",0,0,1,1,116
117,"[{'type': 'Private', 'InvestorBool': True, 'un...",0,0,1,1,117
118,"[{'type': 'Private', 'InvestorBool': True, 'un...",0,0,1,1,118
119,"[{'type': 'Private', 'InvestorBool': True, 'un...",0,0,1,1,119


In [8]:
# post processing 
#extract types of investors from first row
def post_processing(df):
    df2 = df.copy()
    investor_types=[]
    for investor in df2['Investors'][0]:
        investor_types.append(investor['type'])
    #make a new dataframe called df2 with the column names as the investor types and the values as the tokens
    df3 = pd.DataFrame(columns=investor_types)
    for i in range(df2.shape[0]):
        for j in range(Investor_list.__len__()):
            df3.loc[i, df['Investors'][i][j]['type']] = df2['Investors'][i][j]['tokens']

    # remove the investor column from df and add df2 into df)
    df2 = pd.concat([df3, df2['timestep']], axis=1)
    # df2.set_index('timestep', inplace=True)
    df2.rename(columns={'timestep': 'month'}, inplace=True)
    return df2

In [9]:
clean_df = post_processing(df)

In [10]:
from cadcad_gpt import CadCAD_GPT

In [11]:
# read file pred_prey_docs.md
# with open('pred_prey_docs.md', 'r') as file:
#     docs = file.read()

In [12]:
# read the openaikey from .env file
import os
from dotenv import load_dotenv
load_dotenv()
openai_key = os.getenv("OPENAI_KEY")

In [13]:
#research.csv to pandas dataframe
import pandas as pd
research_df = pd.read_csv('research.csv')

In [14]:
research_df

,Project,Round,Date,Amount,Sector
0,M^ZERO,Seed,2023-04-05,$22.50m,DeFi
1,Nibiru,Seed,2023-04-04,$8.50m,DeFi
2,Trident Digital,Seed,2023-09-06,$8.00m,DeFi
3,Ethena,Seed,2023-07-17,$6.50m,DeFi
4,TapiocaDAO,Seed,2023-06-17,$6.00m,DeFi
5,Azteco,Seed,2023-05-18,$6.00m,DeFi
6,Fractal Protocol,Seed,2023-04-17,$6.00m,DeFi
7,Puffer Finance,Seed,2023-08-08,$5.50m,DeFi
8,Cega,Seed,2023-03-27,$5.00m,DeFi
9,Tristero,Seed,2023-05-03,$4.80m,DeFi


In [15]:
cadcad_gpt = CadCAD_GPT(openai_key, model, simulation, experiment, clean_df)

In [16]:
cadcad_gpt('hi')

"Planner Agent didn't make a plan. Please try again or tweak the prompt to a question."

In [17]:
cadcad_gpt('Can you plot the Private column')

"Planner Agent didn't make a plan. Please try again or tweak the prompt to a question."

In [18]:
cadcad_gpt('Which projects raised more than 4 mil in seed round in Defi sector in the last 3 months')

Planner Agent:
I have made a plan to follow:
Step 1 use the function long_term_memory to fetch the projects that raised more than 4 million in the seed round in the Defi sector in the last 3 months


Executor Agent:
Thought: My task is to use the function long_term_memory to fetch the projects that raised more than 4 million in the seed round in the Defi sector in the last 3 months
Action: I should call long_term_memory function with these {'query': "SELECT project, raised, sector, round, date FROM table WHERE raised > 4000000 AND round = 'seed' AND sector = 'Defi' AND date >= CURRENT_DATE - INTERVAL '3 months'"} arguments.
Observation: M^ZERO, Nibiru, Trident Digital


In [19]:
cadcad_gpt("What is the distribution of token in month 12, if the total token supply was 15 mil?")

Planner Agent:
I have made a plan to follow:
Step 1 use the function change_param to set the total_token_supply parameter to 15 million
Step 2 use the function analysis_agent to fetch the distribution of tokens in month 12.


Executor Agent:
Thought: My task is to use the function change_param to set the total_token_supply parameter to 15 million
Action: I should call change_param function with these {'param': 'total_token_supply', 'value': 15000000} arguments.
Observation: new total_token_supply value is 15000000.0 and the simulation dataframe is updated
Executor Agent:
Thought: My task is to use the function analysis_agent to fetch the distribution of tokens in month 12.
Action: I should call analysis_agent function with these {'question': 'What is the distribution of tokens in month 12?'} arguments.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['month'] == 12].sum()"}`


Private      2002498.0
Public        135000.0
Team          125000.

In [20]:
cadcad_gpt('if the total token supply was 15 mil? What would be the distribution of token in month 12?')

Planner Agent:
I have made a plan to follow:
Step 1 use the function change_param to update the total token supply to 15 million
Step 2 use the function analysis_agent to get the token distribution in month 12.


Executor Agent:
Thought: My task is to use the function change_param to update the total token supply to 15 million
Action: I should call change_param function with these {'param': 'total_token_supply', 'value': 15000000} arguments.
Observation: new total_token_supply value is 15000000.0 and the simulation dataframe is updated
Executor Agent:
Thought: My task is to use the function analysis_agent to get the token distribution in month 12.
Action: I should call analysis_agent function with these {'question': 'What is the token distribution in month 12?'} arguments.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['month'] == 12]"}`


      Private    Public      Team Liquidity  Ecosystem  month
12  2002498.0  135000.0  125000.0  750000

In [21]:
cadcad_gpt('hi')

"Planner Agent didn't make a plan. Please try again or tweak the prompt to a question."

In [22]:
from colorama import Fore, Back, Style
string = '''The distribution of tokens in month 12 is as follows:

- Private: 2,002,498 tokens
- Public: 135,000 tokens
- Team: 125,000 tokens
- Liquidity: 750,000 tokens
- Ecosystem: 2,401,663 tokens'''
# print that string with a cyan foreground using colorama and white background
# make it bold
print(Style.BRIGHT + Fore.LIGHTCYAN_EX + string)

The distribution of tokens in month 12 is as follows:

- Private: 2,002,498 tokens
- Public: 135,000 tokens
- Team: 125,000 tokens
- Liquidity: 750,000 tokens
- Ecosystem: 2,401,663 tokens
